In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
with open('authorized.txt', 'w') as a:
    a.write('True')

In [4]:
import streamlit as st
from multipage_template import save, MultiPage, start_app, clear_cache
import pandas as pd
from PIL import Image
import os
from pyspark.sql import functions as f
from pyspark.sql import SparkSession
import random
from sklearn.decomposition import PCA
import math
from spotifyAPI import Spotify

In [5]:
userOneFeatures = pd.read_csv("../userOneFeatures.csv")
userTwoFeatures = pd.read_csv("../userTwoFeatures.csv")
userThreeFeatures = pd.read_csv("../userThreeFeatures.csv")

userOneFeatures.drop('Unnamed: 0', axis = 1, inplace = True)
userTwoFeatures.drop('Unnamed: 0', axis = 1, inplace = True)
userThreeFeatures.drop('Unnamed: 0', axis = 1, inplace = True)

In [32]:
s = Spotify()

Client ID:
········
Client Secret:
········


In [6]:
import json

In [118]:
def get_X(playlist = 'Top 50 - USA', creator = 'Spotify', playlist_id = ""):
    '''
    Takes in a playlis,m ;l909t name and the creator of the playlist and
    returns the list of songs in that playlist. Also returns the features.

    Params:
    playlist (string) - the name of the playlist (default Top 50)
    creator (string) - the creator of the playlist (default Spotify)

    Returns:
    songs (list) - a list of song names
    X (list of dictionaries) - our features
    '''
    if os.path.exists('X_songs.txt'):
        #if our song exists
        songs = open('X_songs.txt').read().split("~")
        if len(songs) <= 1:
            songs = s.get_playlist_items(query = playlist, desired_artist = creator, playlist_id = playlist_id)
            with open('X_songs.txt', 'w', encoding = 'utf-8') as s_file:
                for song in songs:
                    s_file.write(song[0] + "~")
                songs = open('X_songs.txt').read().split("~")
    else:
        songs = s.get_playlist_items(query = playlist, desired_artist = creator, playlist_id = playlist_id)
        with open('X_songs.txt', 'w', encoding = 'utf-8') as s_file:
            for song in songs:
                s_file.write(song[0] + "~")
            songs = open('X_songs.txt').read().split("~")

    if os.path.exists('X_features.json'):
        f = open('X_features.json')
        features = json.load(f)
        if len(features) <= 1:
            features = s.get_playlist_features(query = playlist, desired_artist = creator, playlist_id = playlist_id)
            with open('X_features.json', 'w', encoding='utf-8') as f:
                json.dump(features, f, ensure_ascii=False, indent=4)
    else:
        features = s.get_playlist_features(query = playlist, desired_artist = creator, playlist_id = playlist_id)
        with open('X_features.json', 'w', encoding='utf-8') as f:
            json.dump(features, f, ensure_ascii=False, indent=4)

    return songs[:len(songs)-1], features

def get_y(playlist, creator, playlist_id):
    '''
    Takes in a playlist name and the creator of the playlist and
    returns the list of songs in that playlist. Also returns the features.

    Params:
    playlist (string) - the name of the playlist
    creator (string) - the creator of the playlist

    Returns:
    songs (list) - a list of song names
    features (list of dictionaries) - our y features
    '''
    if os.path.exists('y_songs.txt'):
        #if our song exists
        songs = open('y_songs.txt').read().strip().split("~")
        if len(songs) <= 0:
            songs = s.get_playlist_items(query = playlist, desired_artist = creator, playlist_id = playlist_id)
            with open('y_songs.txt', 'w', encoding = 'utf-8') as s_file:
                for song in songs:
                    s_file.write(song[0] + "~")
                songs = open('y_songs.txt').read().strip().split("~")
    else:
        songs = s.get_playlist_items(query = playlist, desired_artist = creator, playlist_id = playlist_id)
        with open('y_songs.txt', 'w', encoding = 'utf-8') as s_file:
            for song in songs:
                s_file.write(song[0] + "~")
            songs = open('y_songs.txt').read().strip().split("~")

    if os.path.exists('y_features.json'):
        f = open('y_features.json')
        features = json.load(f)
        if len(features) <= 0:
            features = s.get_playlist_features(query = playlist, desired_artist = creator, playlist_id = playlist_id)
            with open('y_features.json', 'w', encoding='utf-8') as f:
                json.dump(features, f, ensure_ascii=False, indent=4)
    else:
        features = s.get_playlist_features(query = playlist, desired_artist = creator, playlist_id = playlist_id)
        with open('y_features.json', 'w', encoding='utf-8') as f:
            json.dump(features, f, ensure_ascii=False, indent=4)

    return songs[:len(songs)-1], features

In [167]:
y_songs, y = get_y('throwback', 'Brian Huang', '1TQsoXwhdWVb1DHwqBOa8O')

In [168]:
X_songs, X = get_X('songs nobody would be mad abt', 'Tatum Maston')

In [171]:
from recommender import songRecommender

In [172]:
model = songRecommender(X, y, X_songs, y_songs)
#you can also flip it to get the top 50 songs and their corresponding matches

In [173]:
X = model.getX()
y = model.getY()

In [177]:
len([i[0] for i in X])

30

In [178]:
len([i[0] for i in y])

100